<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-9-MF-SGD-/mf_sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#rating = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

Mounted at /content/drive


In [3]:
# 데이터 읽어 오기 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

In [4]:
# # train test 분리
# from sklearn.utils import shuffle
# TRAIN_SIZE = 0.75
# ratings = shuffle(ratings, random_state=1)
# cutoff = int(TRAIN_SIZE * len(ratings))
# ratings_train = ratings.iloc[:cutoff]
# ratings_test = ratings.iloc[cutoff:]

In [5]:
# train test 분리
# Rating df의 user_id를 타겟(종속변수, 예측값)으로 train, test stratified split 실시(user_id 기준)
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
ratings_train, ratings_test = train_test_split(x, test_size=0.25, stratify=y, random_state=12)

In [6]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# MF-SGD

In [7]:
##### MF-SGD 추천 알고리즘 >>>>>>>>>>>>>>>
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):  # 클래스 생성시 실행되는 초기화 함수
        self.R = np.array(ratings)    # df ratings를 np.array로 바꿔 self.R에 저장

        item_id_index = []    # 변수 초기화
        index_item_id = []    # 변수 초기화
        for i, one_id in enumerate(ratings):        # df ratings의 각 items에 대해서 아래 작업 수행
            item_id_index.append([one_id, i])       # save id - index 
            index_item_id.append([i, one_id])       # save index - id
        self.item_id_index = dict(item_id_index)    # dict로 변환
        self.index_item_id = dict(index_item_id)    # dict로 변환

        user_id_index = []                          # 사용자에 대해서도 실행
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)    # dict로 변환
        self.index_user_id = dict(index_user_id)

        self.num_users, self.num_items = np.shape(self.R)
        self.K = K                                  # 잠재요인 수
        self.alpha = alpha                          # 학습률
        self.beta = beta                            # 정규화계수
        self.iterations = iterations                # SGD 계산 반복 횟수
        self.verbose = verbose                      # 중간 학습과정 출력여부

    # train set의 RMSE 계산
    def rmse(self):                            # 현재 P,Q로 RMSE 계산
        xs, ys = self.R.nonzero()              # R에서 평점이 있는(NOT NULL) 요소의 인덱스들
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):                            # 평점이 있는 요소(사용자 x, 아이템 y)에 대해:
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)             # 예측값을 예측리스트에,
            self.errors.append(self.R[x, y] - prediction)   # 오차를 오차리스트에 저장
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Predict Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)    # 평점 예측치(r^)의 식에 사용자, 아이템 편향 추가한 식
        # b : 전체평균, b_u[i] : 사용자i 평가경향(bias), b_d[j] : 아이템j 평가경향, 넷째항 : P, Q.T를 내적한 예측평점
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:                # samples의 (user-item-rating) set에 대해 sgd적용
            prediction = self.get_prediction(i, j)
            e = (r - prediction)                    # 오차 = 실제평점 - 예측

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])     # 사용자,아이템의 경향성을 고려된 평점예측식의 편미분식으로 사용자 평가경향 업데이트
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])     # 아이템 평가경향 업데이트

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])   # 정규화항 추가된 평점예측식의 편미분식으로 P행렬 업뎃
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])   # Q 업뎃

    # Test set을 선정 - 분리된 test set을 넘겨받아서 클래스 내부의 test set을 만드는 함수
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i, 0]]   # 현재 사용자의 인덱스를 user_id_index(매핑 리스트)에서 받아옴
            y = self.item_id_index[ratings_test.iloc[i, 1]]   # 현재 아이템의 인덱스를 item_id_index(매핑 리스트)에서 받아옴
            z = ratings_test.iloc[i, 2]                       # 현재 (사용자-아이템)의 평점
            test_set.append([x, y, z])          # 현재 (사용자-아이템-평점)을 test_set에 추가
            self.R[x, y] = 0                    # Setting test set ratings to 0 : MF는 R 전체를 사용해서 학습하기 때문에 test set은 평점을 지움
        self.test_set = test_set                # test_set을 클래스에 저장한다.
        return test_set                         # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)       # 오차(평점 실제값 - 예측평점)의 제곱을 누적한 값이 저장된다.
        return np.sqrt(error/len(self.test_set))          # error를 RMSE로 변환해서 돌려준다.

    # Training 하면서 test set의 정확도를 계산
    def test(self):   # MF모델을 SGD방식으로 훈련하는 핵심 함수이다.
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))   # P행렬을 (평균 0, 표준편차 1/K인 정규분포 난수)로 초기화한다.
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))   # Q행렬 초기화한다.

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)           # 사용자 평가경향을 초기화, array크기는 사용자수(num_users)와 동일하다.
        self.b_d = np.zeros(self.num_items)           # 아이템 평가경향을 초기화한다.
        self.b = np.mean(self.R[self.R.nonzero()])    # 전체 평균을 구해서 저장한다.

        # List of training samples
        rows, columns = self.R.nonzero()              # 평점행렬 R 중에서 평점있는 요소의 인덱스를 가져온다.
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]   
        # SGD를 적용할 대상, 즉 평점이 있는 요소의 인덱스와 평점을 리스트로 만들어 samples에 저장한다.

        # Stochastic gradient descent for given number of iterations
        training_process = []                             # training_process 를 초기화한다. SGD를 한번 실행할 때마다 RMSE가 얼마나 개선되는지를 기록한다.
        for i in range(self.iterations):                  # 지정된 반복 횟수만큼 SGD를 실행한다.
            np.random.shuffle(self.samples)               # samples 를 임의로 섞는다. 출발점에 따라 수렴의 속도가 달라질 수 있기 때문이다.
            self.sgd()                                    # SGD를 실행하는 함수를 호출한다.
            rmse1 = self.rmse()                           # SGD로 P, Q, bu, bd가 업데이트 되었으므로 이에 따른 RMSE를 계산한다.
            rmse2 = self.test_rmse()                      # test set은 별도로 계산한다.
            training_process.append((i+1, rmse1, rmse2))  # 결과를 저장한다.
            if self.verbose:                              # verbose가 True이면 10회 반복마다 중간 결과를 표시한다.
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        prediction = self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
        return prediction

    # Full user-movie rating matrix (모든 사용자의 모든 아이템에 대한 예측치(Full matrix)를 계산해서 돌려준다)
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)
        # $\hat{r}_{ij}=b+bu_i+bd_j+\sum_{k=1}^Kp_{ik}q_{kj}$


In [8]:
# MF클래스 생성 및 학습
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)    # 전체 데이터를 full matrix로 변환한다.
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=400, verbose=True)          # NEW-MF 클래스를 생성한다.
test_set = mf.set_test(ratings_test)    # ratings_test를 테스트 데이터로 지정하도록 set_test() 함수를 호출한다.
result = mf.test()                      # 정해진 파라미터에 따라 MF 훈련과 정확도 계산을 실행한다.

Iteration: 10 ; Train RMSE = 0.9683 ; Test RMSE = 0.9777
Iteration: 20 ; Train RMSE = 0.9436 ; Test RMSE = 0.9584
Iteration: 30 ; Train RMSE = 0.9328 ; Test RMSE = 0.9507
Iteration: 40 ; Train RMSE = 0.9265 ; Test RMSE = 0.9467
Iteration: 50 ; Train RMSE = 0.9225 ; Test RMSE = 0.9444
Iteration: 60 ; Train RMSE = 0.9197 ; Test RMSE = 0.9430
Iteration: 70 ; Train RMSE = 0.9175 ; Test RMSE = 0.9420
Iteration: 80 ; Train RMSE = 0.9156 ; Test RMSE = 0.9414
Iteration: 90 ; Train RMSE = 0.9138 ; Test RMSE = 0.9408
Iteration: 100 ; Train RMSE = 0.9119 ; Test RMSE = 0.9402
Iteration: 110 ; Train RMSE = 0.9095 ; Test RMSE = 0.9395
Iteration: 120 ; Train RMSE = 0.9062 ; Test RMSE = 0.9384
Iteration: 130 ; Train RMSE = 0.9015 ; Test RMSE = 0.9367
Iteration: 140 ; Train RMSE = 0.8949 ; Test RMSE = 0.9343
Iteration: 150 ; Train RMSE = 0.8862 ; Test RMSE = 0.9312
Iteration: 160 ; Train RMSE = 0.8756 ; Test RMSE = 0.9277
Iteration: 170 ; Train RMSE = 0.8636 ; Test RMSE = 0.9244
Iteration: 180 ; Train 

In [9]:
# iteration = 200
# learning_rate = 0.01
# lmbda = 0.01